In [147]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzysearch import find_near_matches


In [193]:
train = pd.read_csv('refined.csv')
manufacturers = pd.read_csv("manufacturer_list.csv")
print(train.head())
print(manufacturers.head())

                                    article_name
0                    Chicken Burger Patty (250G)
1              BRITANNIA DAILY FAMILY LONG BREAD
2  POOOF POTATO KETTLE CHIPS ZESTY MASALA PP 30g
3                            FEVIBOND TUBE 20 ML
4             KITKAT DESSERT DELIGHT TRUFFLE 50g
         brand_name
0     ORGANIC INDIA
1  head & shoulders
2          Sunfeast
3            Unibic
4           CREMICA


In [194]:
train['transformed'] = train['article_name'].str.lower()
train['transformed'] = train['transformed'].str.replace("'","")
train['transformed'] = train['transformed'].str.replace("(","")
train['transformed'] = train['transformed'].str.replace(")","")
train['transformed'] = train['transformed'].str.replace(")","")
train['transformed'] = train['transformed'].str.replace("/"," ")
train['transformed'] = train['transformed'].str.replace("-","")

In [195]:
train.head(10)

,article_name,transformed
0,Chicken Burger Patty (250G),chicken burger patty 250g
1,BRITANNIA DAILY FAMILY LONG BREAD,britannia daily family long bread
2,POOOF POTATO KETTLE CHIPS ZESTY MASALA PP 30g,pooof potato kettle chips zesty masala pp 30g
3,FEVIBOND TUBE 20 ML,fevibond tube 20 ml
4,KITKAT DESSERT DELIGHT TRUFFLE 50g,kitkat dessert delight truffle 50g
5,BLUE BIRD SPAGHETTI PASTA 200G,blue bird spaghetti pasta 200g
6,NIRAPARA AVALOSE PODI 500GM,nirapara avalose podi 500gm
7,NIVEA ROLL ON FRESH NATURAL 50ml,nivea roll on fresh natural 50ml
8,FIAMA DW SOAP EXOTIC DREAM BX 115g,fiama dw soap exotic dream bx 115g
9,SANGIS KTCN PERI PERI MAYONNAISE BT 200g,sangis ktcn peri peri mayonnaise bt 200g


In [196]:
train['quantity'] = train['transformed'].str.extract(r'([0-9].*(ml|g|kg|l|s|p))')[0]
#train['quantity'] = train['transformed'].str.extract(r'([0-9].*(ml|g|kg|l|s|p))')

In [197]:
train.head()

,article_name,transformed,quantity
0,Chicken Burger Patty (250G),chicken burger patty 250g,250g
1,BRITANNIA DAILY FAMILY LONG BREAD,britannia daily family long bread,NaN
2,POOOF POTATO KETTLE CHIPS ZESTY MASALA PP 30g,pooof potato kettle chips zesty masala pp 30g,30g
3,FEVIBOND TUBE 20 ML,fevibond tube 20 ml,20 ml
4,KITKAT DESSERT DELIGHT TRUFFLE 50g,kitkat dessert delight truffle 50g,50g


In [199]:
train_small = train.head(100)
manufacturers['brand_name'] = manufacturers['brand_name'].str.lower()
manufacturers['brand_name'] = manufacturers['brand_name'].str.replace("'","")
manufacturers['brand_name'] = manufacturers['brand_name'].str.replace("(","")
manufacturers['brand_name'] = manufacturers['brand_name'].str.replace(")","")
manufacturers['brand_name'] = manufacturers['brand_name'].str.replace(")","")
manufacturers['brand_name'] = manufacturers['brand_name'].str.replace("/"," ")
manufacturers['brand_name'] = manufacturers['brand_name'].str.replace("-","")
manufacturers.head()


,brand_name
0,organic india
1,head & shoulders
2,sunfeast
3,unibic
4,cremica


In [200]:
def get_brand_name(article_name, choices):
    brand_name, score, _ = process.extractOne(article_name, choices)
    return brand_name, score

def get_strict(article_name, choices):
    matches = []
    for choice in choices:
        choice1 = choice + " "
        choice2 = " " + choice + " "
        choice3 = " " + choice
        if article_name.startswith(choice1) or article_name.endswith(choice3) or choice2 in article_name:
f            matches.append(choice)
    return matches
            

In [201]:
train_small['strict'] = train_small['transformed'].apply(lambda x: get_strict(x, manufacturers['brand_name']))


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [202]:
train_small


,article_name,transformed,quantity,strict
0,Chicken Burger Patty (250G),chicken burger patty 250g,250g,[]
1,BRITANNIA DAILY FAMILY LONG BREAD,britannia daily family long bread,NaN,[britannia]
2,POOOF POTATO KETTLE CHIPS ZESTY MASALA PP 30g,pooof potato kettle chips zesty masala pp 30g,30g,[]
3,FEVIBOND TUBE 20 ML,fevibond tube 20 ml,20 ml,[fevibond]
4,KITKAT DESSERT DELIGHT TRUFFLE 50g,kitkat dessert delight truffle 50g,50g,[]
5,BLUE BIRD SPAGHETTI PASTA 200G,blue bird spaghetti pasta 200g,200g,[blue bird]
6,NIRAPARA AVALOSE PODI 500GM,nirapara avalose podi 500gm,500g,[nirapara]
7,NIVEA ROLL ON FRESH NATURAL 50ml,nivea roll on fresh natural 50ml,50ml,[nivea]
8,FIAMA DW SOAP EXOTIC DREAM BX 115g,fiama dw soap exotic dream bx 115g,115g,[]
9,SANGIS KTCN PERI PERI MAYONNAISE BT 200g,sangis ktcn peri peri mayonnaise bt 200g,200g,[]


In [209]:
train_small['fuzzy'] = train_small['transformed'].apply(lambda x: get_brand_name(x, manufacturers['brand_name']))


In [210]:
train_small = train_small.filter(['article_name','transformed', 'quantity', 'strict']).join(train_small.fuzzy.apply(lambda loc: pd.Series(loc, index=['fuzzy_brand', 'score'])))


In [211]:
train_small

,article_name,transformed,quantity,strict,fuzzy_brand,score
0,Chicken Burger Patty (250G),chicken burger patty 250g,250g,[],chiko,72
1,BRITANNIA DAILY FAMILY LONG BREAD,britannia daily family long bread,NaN,[britannia],britannia,90
2,POOOF POTATO KETTLE CHIPS ZESTY MASALA PP 30g,pooof potato kettle chips zesty masala pp 30g,30g,[],kettle studio,86
3,FEVIBOND TUBE 20 ML,fevibond tube 20 ml,20 ml,[fevibond],fevibond,90
4,KITKAT DESSERT DELIGHT TRUFFLE 50g,kitkat dessert delight truffle 50g,50g,[],set wet,64
5,BLUE BIRD SPAGHETTI PASTA 200G,blue bird spaghetti pasta 200g,200g,[blue bird],blue bird,90
6,NIRAPARA AVALOSE PODI 500GM,nirapara avalose podi 500gm,500g,[nirapara],nirapara,90
7,NIVEA ROLL ON FRESH NATURAL 50ml,nivea roll on fresh natural 50ml,50ml,[nivea],nivea,90
8,FIAMA DW SOAP EXOTIC DREAM BX 115g,fiama dw soap exotic dream bx 115g,115g,[],fiama di wills,86
9,SANGIS KTCN PERI PERI MAYONNAISE BT 200g,sangis ktcn peri peri mayonnaise bt 200g,200g,[],sangis kitchen,86
